In [ ]:
from crimson.folder_watcher import create_watcher
import re


# Basic usage - simple file copying
watcher = create_watcher(
    source_dir="./source", 
    output_dir="./output",
    include=["*.py", "*.md"],   # Only process Python and markdown files
    exclude=["*_test.py"],      # Exclude test files
    process_existing=False,     # Don't process existing files
    auto_start=False            # Don't start immediately
)

# Example file editor registration
def add_timestamp_to_python(src_path, content):
    """Add timestamp to the top of Python files"""
    import datetime
    timestamp = datetime.datetime.now().strftime("# Last modified: %Y-%m-%d %H:%M:%S\n")
    
    # Check if timestamp already exists
    if "# Last modified:" in content.split("\n")[0]:
        # Replace first line
        lines = content.split("\n")
        lines[0] = timestamp.strip()
        return "\n".join(lines)
    else:
        # Add to top of file
        return timestamp + content

# Register editor for all Python files to add timestamp
watcher.register_editor("*.py", add_timestamp_to_python)


def md_to_story_editor(src_path: str, content: str):
    # Generate storybook file content based on markdown file path
    # File content is not actually used but needed to maintain interface

    titlePrefix = 'AutoPost'
    componentPath = '../components/Page'
    postPath = '../../post' 

    path_parts = src_path.split('/')
    file_name = path_parts[-1].replace('.md', '')
    normalized_file_name = re.sub(r'[^a-zA-Z0-9]', '_', file_name)
    title_path = '/'.join([titlePrefix] + path_parts[:-1])
    md_path = postPath + src_path

    # Generate storybook file content
    story_content = f"""
// Auto-generated story file for {src_path}
import React from 'react';
import {{ Page }} from '{componentPath}';
import markdownContent from '{md_path}?raw';

export default {{
    title: '{title_path}',
    parameters: {{}},
}};

export const {normalized_file_name} = () => {{
    return Page(markdownContent);
}};
"""

    return story_content, src_path.replace('.md', '.stories.tsx')

# Register editor for markdown files to convert them to stories
watcher.register_editor("*.md", md_to_story_editor)

In [2]:
watcher.start()


Watching './source' for changes...


In [3]:
watcher.process_existing_files()

Processing existing files...
Edited and saved './source/my_post.md' to './output/my_post.stories.tsx'
Edited and saved './source/any_script.py' to './output/any_script.py'
Processed 2 existing files.


In [4]:
watcher.stop()

Stopped watching for changes.


Check the result

Source files:
- [my_post.md](./source/my_post.md)
- [any_script.py](./source/any_script.py)

Output files:
- [my_post.stories.tsx](./output/my_post.stories.tsx)
- [any_script.py](./output/any_script.py)